In [28]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from collections import Counter

# nlp = spacy.load('en_core_web_sm')
# nlpmd = spacy.load('en_core_web_md')
nlplg = spacy.load('en_core_web_lg')


In [29]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content": raw_data, # msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [30]:
# data
data["content"] = data["content"].apply(lambda x: x.replace('\n', ' '))
# data['content']

In [31]:
body = data["content"].values
entityData = []
for cnt in body:
    entityCh = []
    entity = nlplg(cnt)
    for token in entity.ents:
        entityData.append({"name":token.text, "label":token.label_, "end_char":token.end_char,"start_char":token.start_char,"text":cnt})


#     entityData.append(entityCh)


In [62]:
# body

In [34]:
import spacy
from pathlib import Path
import json

texts = body  # your corpus
label = 'NEW_LABEL'  # the label you want to assign to the patterns
patterns = []  # collect patterns here

nlp = spacy.load('en_core_web_sm')  # or any other model
docs = nlp.pipe(texts)  # use nlp.pipe for efficiency
for doc in docs:
    for ent in doc.ents:
        if ent.label_ == 'DATE':   # if a DATE entity is found
            entry = {'label': label, "text":doc, 'pattern': [{'lower': ent.text}]}
            patterns.append(entry)

# dump JSON and write patterns to file
#jsonl = [json.dumps(pattern) for pattern in patterns]
#Path('patterns.jsonl').open('w', encoding='utf-8').write('\n'.join(jsonl))

In [37]:
# Path('patterns.jsonl').open('r', encoding='utf-8').read()

In [53]:
your_string = """i want to book a room        at Lake         Palace.

Check in date is -       31st December 2018
check out - 1st       January 2019"""

In [81]:
def text_preprocessing(text):
    return ' '.join(text.replace('\n', ' ').replace('1st', '1 st').replace('2nd', '2 nd').replace('3rd', '3 rd').split())

In [57]:
your_string = your_string.replace('1st', '1st')
your_string =" ".join(your_string.strip().split())

In [85]:
doc = nlp(u""+your_string)

In [86]:
for ent in doc.ents:
    print(ent)

Lake Palace
December 2018
January 2019


In [61]:
your_string.replace('1st', '1st').replace

<function str.replace>

In [87]:
ent = nlplg(u""+text_preprocessing(your_string))

In [88]:
for dt in ent.ents:
    print(dt)

Lake Palace
December 2018
1 st January 2019


In [90]:
# doc

In [92]:
date_pattern = [{"example": ["September 30, 1971", "September 30 1971"], "pattern": [{"ENT_TYPE": "DATE", "IS_ALPHA": True}, {"ENT_TYPE": "DATE", "IS_DIGIT": True}, {"ENT_TYPE": "DATE", "ORTH": ",", "OP": "*"}, {"ENT_TYPE": "DATE", "IS_DIGIT": True}], "label": "MY_DATE"},
{"example": ["30 September, 1971", "30 September 1971"], "pattern": [{"ENT_TYPE": "DATE", "IS_DIGIT": True}, {"ENT_TYPE": "DATE", "IS_ALPHA": True}, {"ENT_TYPE": "DATE", "ORTH": ",", "OP": "*"}, {"ENT_TYPE": "DATE", "IS_DIGIT": True}], "label": "MY_DATE"},
{"example": ["1st day of September, 1971"], "pattern": [{"SHAPE": "dxx"}, {"LOWER": "day"}, {"LOWER": "of"}, {"ENT_TYPE": "DATE", "IS_ALPHA": True}, {"ENT_TYPE": "DATE", "ORTH": ",", "OP": "*"}, {"ENT_TYPE": "DATE", "IS_DIGIT": True}], "label": "MY_DATE"},
{"example": ["30th day of September, 1971"], "pattern": [{"SHAPE": "ddxx"}, {"LOWER": "day"}, {"LOWER": "of"}, {"ENT_TYPE": "DATE", "IS_ALPHA": True}, {"ENT_TYPE": "DATE", "ORTH": ",", "OP": "*"}, {"ENT_TYPE": "DATE", "IS_DIGIT": True}], "label": "MY_DATE"},
{"example": ["1/1/1971"], "pattern": [{"SHAPE": "d/d/dddd"}], "label": "MY_DATE"},
{"example": ["10/1/1971"], "pattern": [{"SHAPE": "dd/d/dddd"}], "label": "MY_DATE"},
{"example": ["1/10/1971"], "pattern": [{"SHAPE": "d/dd/dddd"}], "label": "MY_DATE"},
{"example": ["10/10/1971"], "pattern": [{"SHAPE": "dd/dd/dddd"}], "label": "MY_DATE"},
{"example": ["1/1/71"], "pattern": [{"SHAPE": "d/d/dd"}], "label": "MY_DATE"},
{"example": ["10/1/71"], "pattern": [{"SHAPE": "dd/d/dd"}], "label": "MY_DATE"},
{"example": ["1/10/71"], "pattern": [{"SHAPE": "d/dd/dd"}], "label": "MY_DATE"},
{"example": ["10/10/71"], "pattern": [{"SHAPE": "dd/dd/dd"}], "label": "MY_DATE"}]

In [93]:
date_pattern

[{'example': ['September 30, 1971', 'September 30 1971'],
  'pattern': [{'ENT_TYPE': 'DATE', 'IS_ALPHA': True},
   {'ENT_TYPE': 'DATE', 'IS_DIGIT': True},
   {'ENT_TYPE': 'DATE', 'ORTH': ',', 'OP': '*'},
   {'ENT_TYPE': 'DATE', 'IS_DIGIT': True}],
  'label': 'MY_DATE'},
 {'example': ['30 September, 1971', '30 September 1971'],
  'pattern': [{'ENT_TYPE': 'DATE', 'IS_DIGIT': True},
   {'ENT_TYPE': 'DATE', 'IS_ALPHA': True},
   {'ENT_TYPE': 'DATE', 'ORTH': ',', 'OP': '*'},
   {'ENT_TYPE': 'DATE', 'IS_DIGIT': True}],
  'label': 'MY_DATE'},
 {'example': ['1st day of September, 1971'],
  'pattern': [{'SHAPE': 'dxx'},
   {'LOWER': 'day'},
   {'LOWER': 'of'},
   {'ENT_TYPE': 'DATE', 'IS_ALPHA': True},
   {'ENT_TYPE': 'DATE', 'ORTH': ',', 'OP': '*'},
   {'ENT_TYPE': 'DATE', 'IS_DIGIT': True}],
  'label': 'MY_DATE'},
 {'example': ['30th day of September, 1971'],
  'pattern': [{'SHAPE': 'ddxx'},
   {'LOWER': 'day'},
   {'LOWER': 'of'},
   {'ENT_TYPE': 'DATE', 'IS_ALPHA': True},
   {'ENT_TYPE': 'D